In [2]:
import pandas as pd
import requests
import json
import glob
import os
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [3]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsicmF0IjoxNTU0ODg1OTk2LCJ0aGlua19pZCI6IjE0MTIzMjMxIiwiY3VzdG9tZXJfcHJvZHVjdF9ob2xkaW5nIjpbIlpaX05MX1JFRyIsIlpaX1NDX0REQUZMIl0sInNpdGUiOiJzdXBlcmNvYWNoX2hlcmFsZHN1biIsImhhc19jb25zZW50ZWRfdG9fdGMiOnRydWUsImF0c19oYXNoIjoiZjZmMjQ2NGUwNGFiNzgzYWJjN2FkYTA3ZDIwZTZmYjMwNmExOWRiNjYzMjVmYjY4ZWRhOGQ1ODNiYThkNTk4ZS41NmNmOTgyMzcyMmM1MDEwYzE5YjdiOTIwMWZmOGZjZTUzNjc2N2ExIiwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMy0wMi0yN1QxMDo0MjozNi45ODFaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE2Nzc0OTQ1NTcsImV4cCI6MTY4NDgzODU1Nywic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXRfaGFzaCI6ImptVGllTUZNQnA5Uk5UdjBBMjFTQUEiLCJzaWQiOiJHZW92T0hiTlRNS1d1NmtzbjVmaHB4Q3d2SFNwRVNNbCIsIm5vbmNlIjoib1BybXB3STkuM1lNUXkuVk9lSEN3eTI4RVF5RkRqdnoifQ.HFUNalMK1HiF6sLePKMQ96aAkjxhUTzQL2vpRv7j7m8TNu3l9CZoUyCtSB7Wz1kqv3QNrxUflPCJ2DPMFyE87HaAolms0g041AuqQYYNViWXy8hE-5yWrlrkKi1ZJXIotIaVYga7IK-5dqB_RJhEysZqt1Tx9-bmaMINkKq4LHdk4wbbxamvDuPtcoC-09_5vIIHZlSdHuGFMnBTs4skki5L7QWBhJARMKmS5-LkvACiO7jirvh1z3imsQ2w5ZG_2jqnxmU3TW1mE-mr2DIHpNa17LrUq0pjZ6nln1mH1Yydb8lAy-KIR99tTdAbwQ8jXK3x456nENKqZ-R4lyGSIw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2022/api/afl/classic/v1/access_token", data=payload, verify=False)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"message":"Internal server error","status_code":500}'

KeyError: 'access_token'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,714,295467,Jack,Macrae,WBD,MID
0,643,296205,Jack,Steele,STK,MID
0,292,298272,Touk,Miller,GCS,MID
0,463,996701,Clayton,Oliver,MEL,MID
0,447,290528,Max,Gawn,MEL,RUC
...,...,...,...,...,...,...
0,201,294092,Anthony,*** McDonald-Tipungwuti,ESS,FWD
0,131,998167,Tom,*** Williamson,CAR,DEF
0,619,260113,Jarryn,*** Geary,STK,MID
0,444,290199,Majak,*** Daw,MEL,DEF RUC


## Player List with Averages

In [5]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/statspack?season=2022&round=99", headers=headers) 
player_json = json.loads(s.text)
df_player_list_averages = pd.DataFrame()

for player in player_json:
    df_line = pd.DataFrame(
        {
            'player_id': [player['player_id']],
            'total_games' : [player['total_games']],
            'total_points': [player['total_points']]
        }
    )
    df_player_list_averages = pd.concat([df_player_list_averages, df_line])
df_player_list_averages['average'] = df_player_list_averages['total_points'] / df_player_list_averages['total_games']
df_player_list_averages

,player_id,total_games,total_points,average
0,1,17,782,46.000000
0,2,0,0,NaN
0,3,11,657,59.727273
0,4,22,1252,56.909091
0,5,3,144,48.000000
...,...,...,...,...
0,785,6,209,34.833333
0,786,0,0,NaN
0,789,3,177,59.000000
0,791,1,13,13.000000


In [6]:
df_player_list_averages.to_csv('outputs/player_list_averages.csv')

## LADDER AND FIXTURE DATA
- also saves each json file

In [7]:
# specify round
round_number = 18
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/leagues/651660/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)
    with open("outputs/json files/match_json_rd{}.json".format(rd), 'w') as outfile:
        json.dump(match_json, outfile, indent=4, sort_keys=True)

## COACH IDs AND NAMES

In [8]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,10883,Lester,Love is Hind
0,21562,10834,Mark,Dunkley's Donuts
0,45168,14003,Luke,Gov Is In The Air
0,8258,11125,Simon,Sloane Andreas
0,300791,5535,Paul,We're the Millers
0,71052,10882,James,Take Me To Parish
0,180650,63015,Jordan,Wolf of Walsh St
0,10857,10836,Anthony,Cumming for the DB


In [9]:
df_coaches.to_csv('inputs/coach_list.csv')

## PLAYER MATCH RESULTS

Function to get player scores for each match

In [10]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [11]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,86078,10883,Lester,Love is Hind,368,MID,164,True,1009528,Tom,Green,GWS,MID,R1
1,2,86078,10883,Lester,Love is Hind,368,MID,106,True,1009528,Tom,Green,GWS,MID,R2
2,3,86078,10883,Lester,Love is Hind,368,MID,147,True,1009528,Tom,Green,GWS,MID,R3
3,4,86078,10883,Lester,Love is Hind,368,MID,92,True,1009528,Tom,Green,GWS,MID,R4
4,5,86078,10883,Lester,Love is Hind,368,MID,115,True,1009528,Tom,Green,GWS,MID,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,18,45168,14003,Luke,Gov Is In The Air,266,DEF,45,True,998260,Jack,Bowes,GCS,DEF,R18
3308,16,10857,10836,Anthony,Cumming for the DB,708,MID,49,True,294557,Lachie,Hunter,WBD,MID,R16
3309,17,8258,11125,Simon,Sloane Andreas,708,MID,63,True,294557,Lachie,Hunter,WBD,MID,R17
3310,17,71052,10882,James,Take Me To Parish,152,MID,85,True,291720,Will,Hoskin-Elliott,COL,FWD MID,R17


In [12]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [13]:
df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,86078,10883,Lester,Love is Hind,1,0,0,1931,1683,4,1
0,1,21562,10834,Mark,Dunkley's Donuts,1,0,0,1841,1727,4,2
0,1,45168,14003,Luke,Gov Is In The Air,1,0,0,1752,1554,4,3
0,1,8258,11125,Simon,Sloane Andreas,1,0,0,1692,1553,4,4
0,1,300791,5535,Paul,We're the Millers,0,0,1,1727,1841,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,18,21562,10834,Mark,Dunkley's Donuts,10,0,8,30886,29853,40,4
0,18,71052,10882,James,Take Me To Parish,9,0,9,29490,30418,36,5
0,18,10857,10836,Anthony,Cumming for the DB,8,0,10,30055,30228,32,6
0,18,180650,63015,Jordan,Wolf of Walsh St,7,0,11,29014,30246,28,7


In [14]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [15]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,5535,We're the Millers,Paul,1727,10834,Dunkley's Donuts,Mark,1841
0,1,Anthony vs Simon,10836,Cumming for the DB,Anthony,1553,11125,Sloane Andreas,Simon,1692
0,1,James vs Lester,10882,Take Me To Parish,James,1683,10883,Love is Hind,Lester,1931
0,1,Luke vs Jordan,14003,Gov Is In The Air,Luke,1752,63015,Wolf of Walsh St,Jordan,1554
0,2,Paul vs Anthony,5535,We're the Millers,Paul,1742,10836,Cumming for the DB,Anthony,1757
...,...,...,...,...,...,...,...,...,...,...
0,17,Lester vs Simon,10883,Love is Hind,Lester,1812,11125,Sloane Andreas,Simon,1741
0,18,Paul vs Luke,5535,We're the Millers,Paul,1794,14003,Gov Is In The Air,Luke,1714
0,18,Jordan vs James,63015,Wolf of Walsh St,Jordan,1476,10882,Take Me To Parish,James,1633
0,18,Lester vs Anthony,10883,Love is Hind,Lester,1942,10836,Cumming for the DB,Anthony,1763


In [16]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [17]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,James vs Lester,10883,Love is Hind,Lester,1931,10882,Take Me To Parish,James,1683,1.0,6.0
0,1,Paul vs Mark,10834,Dunkley's Donuts,Mark,1841,5535,We're the Millers,Paul,1727,2.0,4.0
0,1,Luke vs Jordan,14003,Gov Is In The Air,Luke,1752,63015,Wolf of Walsh St,Jordan,1554,3.0,7.0
0,1,Paul vs Mark,5535,We're the Millers,Paul,1727,10834,Dunkley's Donuts,Mark,1841,4.0,2.0
0,1,Anthony vs Simon,11125,Sloane Andreas,Simon,1692,10836,Cumming for the DB,Anthony,1553,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,18,Lester vs Anthony,10836,Cumming for the DB,Anthony,1763,10883,Love is Hind,Lester,1942,4.0,1.0
0,18,Paul vs Luke,14003,Gov Is In The Air,Luke,1714,5535,We're the Millers,Paul,1794,5.0,3.0
0,18,Jordan vs James,10882,Take Me To Parish,James,1633,63015,Wolf of Walsh St,Jordan,1476,6.0,8.0
0,18,Simon vs Mark,11125,Sloane Andreas,Simon,1608,10834,Dunkley's Donuts,Mark,1795,7.0,2.0


## CURRENTLY LISTED TEAMS

In [18]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/leagues/651660/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,714,owner,5535.0,NaN,295467,Jack,Macrae,WBD,MID,300791,5535,Paul,We're the Millers
1,643,owner,10882.0,NaN,296205,Jack,Steele,STK,MID,71052,10882,James,Take Me To Parish
2,292,owner,5535.0,NaN,298272,Touk,Miller,GCS,MID,300791,5535,Paul,We're the Millers
3,463,owner,5535.0,NaN,996701,Clayton,Oliver,MEL,MID,300791,5535,Paul,We're the Millers
4,447,owner,14003.0,NaN,290528,Max,Gawn,MEL,RUC,45168,14003,Luke,Gov Is In The Air
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,96,owner,14003.0,NaN,996731,Charlie,Curnow,CAR,FWD,45168,14003,Luke,Gov Is In The Air
180,226,owner,5535.0,NaN,998129,Will,Brodie,FRE,MID FWD,300791,5535,Paul,We're the Millers
181,384,owner,10834.0,NaN,992644,Braydon,Preuss,GWS,RUC,21562,10834,Mark,Dunkley's Donuts
182,143,owner,10883.0,NaN,1023261,Nick,Daicos,COL,DEF MID,86078,10883,Lester,Love is Hind


## CURRENT FREE AGENTS + WAIVER

In [19]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position
20,488,free_agent,NaN,NaN,280921,Ben,Cunnington,NTH,MID
23,520,free_agent,NaN,NaN,280944,Jack,Ziebell,NTH,DEF FWD
36,280,free_agent,NaN,NaN,993799,Brayden,Fiorini,GCS,MID
43,243,free_agent,NaN,NaN,240052,David,Mundy,FRE,MID
54,37,free_agent,NaN,NaN,270963,Rory,Sloane,ADE,MID
...,...,...,...,...,...,...,...,...,...
805,201,free_agent,NaN,NaN,294092,Anthony,*** McDonald-Tipungwuti,ESS,FWD
806,131,free_agent,NaN,NaN,998167,Tom,*** Williamson,CAR,DEF
807,619,free_agent,NaN,NaN,260113,Jarryn,*** Geary,STK,MID
808,444,free_agent,NaN,NaN,290199,Majak,*** Daw,MEL,DEF RUC


In [20]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [21]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/leagues/651660/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "Waiver"
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,10834,2022-03-22T04:00:03+11:00,502,296355,Luke,McDonald,NTH,102,999827,Zac,Fisher,CAR,Waiver
1,2,10836,2022-03-22T04:00:03+11:00,683,1006126,James,Rowbottom,SYD,664,997142,Peter,Ladhams,SYD,Waiver
2,2,63015,2022-03-22T04:00:03+11:00,618,294307,Brad,Crouch,STK,512,1002240,Jaidyn,Stephenson,NTH,Waiver
3,2,5535,2022-03-22T04:00:03+11:00,719,1002404,Aaron,Naughton,WBD,436,1002222,James,Worpel,HAW,Waiver
4,2,14003,2022-03-22T04:00:03+11:00,336,298419,Brandan,Parfitt,GEE,52,990609,Charlie,Cameron,BRL,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,19,10883,2022-07-19T04:00:02+10:00,533,1000972,Willem,Drew,PTA,58,298211,Darcy,Fort,BRL,Waiver
157,19,10882,2022-07-19T04:00:02+10:00,610,296351,Jack,Billings,STK,152,291720,Will,Hoskin-Elliott,COL,Waiver
158,19,5535,2022-07-19T04:00:02+10:00,512,1002240,Jaidyn,Stephenson,NTH,469,296359,Christian,Salem,MEL,Waiver
159,19,10882,2022-07-19T04:00:02+10:00,67,993828,Rhys,Mathieson,BRL,453,1013409,James,Jordon,MEL,Waiver


In [22]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [23]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/leagues/651660/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "Free Agent"
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev,source
0,1,5535,2022-03-15T07:43:25+11:00,158,1003130,Patrick,Lipinski,COL,9.0,1000932,Tom,Doedee,ADE,Free Agent
1,1,5535,2022-03-18T07:56:29+11:00,774,998128,Alex,Witherden,WCE,330.0,294199,Sam,Menegola,GEE,Free Agent
2,2,5535,2022-03-22T06:00:46+11:00,651,1006028,Nick,Blakey,SYD,565.0,1000223,Liam,Baker,RIC,Free Agent
3,2,5535,2022-03-22T06:02:21+11:00,384,992644,Braydon,Preuss,GWS,38.0,291748,Brodie,Smith,ADE,Free Agent
4,2,5535,2022-03-26T08:40:01+11:00,541,261396,Robbie,Gray,PTA,384.0,992644,Braydon,Preuss,GWS,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,14,63015,2022-06-16T19:08:28+10:00,680,290188,Sam,Reid,SYD,116.0,297907,Nic,Newman,CAR,Free Agent
255,14,63015,2022-06-15T08:18:55+10:00,38,291748,Brodie,Smith,ADE,526.0,994599,Riley,Bonner,PTA,Free Agent
256,16,63015,2022-06-28T04:30:20+10:00,367,993902,Matt,Flynn,GWS,271.0,994077,Mabior,Chol,GCS,Free Agent
257,17,63015,2022-07-05T06:27:14+10:00,625,295584,Bradley,Hill,STK,96.0,996731,Charlie,Curnow,CAR,Free Agent


In [24]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [25]:
s = session.get("https://supercoach.heraldsun.com.au/2022/api/afl/draft/v1/leagues/651660/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)

In [26]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'Trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

,id,trade_id,player_id,trade_player_id,round,user_team_id,processed,status,source
0,63845,40971,767,713,5,10883,2022-04-14T18:59:16+10:00,7,Trade
1,63846,40971,6,0,5,10883,2022-04-14T18:59:16+10:00,7,Trade
0,74455,47581,568,65,6,5535,2022-04-22T19:41:58+10:00,7,Trade
1,74456,47581,664,0,6,5535,2022-04-22T19:41:58+10:00,7,Trade
0,84433,53880,183,319,7,5535,2022-04-30T08:52:56+10:00,2,Trade
...,...,...,...,...,...,...,...,...,...
0,142804,88529,316,494,17,5535,2022-07-07T15:22:01+10:00,2,Trade
1,142805,88529,384,0,17,5535,2022-07-07T15:22:01+10:00,2,Trade
0,142838,88542,316,494,17,5535,2022-07-07T15:44:42+10:00,7,Trade
1,142839,88542,384,0,17,5535,2022-07-07T15:44:42+10:00,7,Trade


In [27]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

,round,user_team_id,processed,player_id,source,player.feed_id,player.first_name,player.last_name,player.team.abbrev
0,5,10883,2022-04-14T18:59:16+10:00,767,Trade,280078,Luke,Shuey,WCE
1,5,10883,2022-04-14T18:59:16+10:00,6,Trade,297401,Matt,Crouch,ADE
2,5,10883,2022-04-14T18:59:16+10:00,713,Trade,290799,Tom,Liberatore,WBD
3,5,10883,2022-04-14T18:59:16+10:00,0,Trade,NaN,NaN,NaN,NaN
4,6,5535,2022-04-22T19:41:58+10:00,568,Trade,993993,Shai,Bolton,RIC
5,6,5535,2022-04-22T19:41:58+10:00,664,Trade,997142,Peter,Ladhams,SYD
6,6,5535,2022-04-22T19:41:58+10:00,65,Trade,293716,Jarryd,Lyons,BRL
7,6,5535,2022-04-22T19:41:58+10:00,0,Trade,NaN,NaN,NaN,NaN
8,8,5535,2022-05-06T20:17:06+10:00,243,Trade,240052,David,Mundy,FRE
9,8,5535,2022-05-06T20:17:06+10:00,6,Trade,297401,Matt,Crouch,ADE


## Combined Free Agents, Waiver, and Trades transactions list

In [28]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
218,1,63015,2022-03-15T05:17:09+11:00,391,1005247,Sam,Taylor,GWS,719.0,1002404,Aaron,Naughton,WBD,Free Agent
115,1,10883,2022-03-15T06:16:27+11:00,762,281373,Jack,Redden,WCE,686.0,294737,Callum,Sinclair,SYD,Free Agent
116,1,10883,2022-03-15T07:28:24+11:00,108,1001398,Matthew,Kennedy,CAR,762.0,281373,Jack,Redden,WCE,Free Agent
0,1,5535,2022-03-15T07:43:25+11:00,158,1003130,Patrick,Lipinski,COL,9.0,1000932,Tom,Doedee,ADE,Free Agent
46,1,10836,2022-03-15T09:11:09+11:00,657,240399,Lance,Franklin,SYD,498.0,1011640,Jason,Horne-Francis,NTH,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,19,14003,2022-07-19T04:23:01+10:00,116,297907,Nic,Newman,CAR,266.0,998260,Jack,Bowes,GCS,Free Agent
215,19,14003,2022-07-19T04:28:21+10:00,737,990290,Tom,Barrass,WCE,116.0,297907,Nic,Newman,CAR,Free Agent
216,19,14003,2022-07-19T04:30:00+10:00,424,1020895,Jai,Newcombe,HAW,49.0,1002312,Zac,Bailey,BRL,Free Agent
27,19,5535,2022-07-20T07:34:33+10:00,122,298290,Marc,Pittonet,CAR,512.0,1002240,Jaidyn,Stephenson,NTH,Free Agent


In [29]:
df_transactions.to_csv('outputs/transactions.csv')

## Add source to player match results

In [30]:
def get_transaction_status(row):
    post_transaction_list = pd.DataFrame()
    round_number = row['round']
    feed_id = row['player.feed_id']
    source = row['source']
    player_rows = df_player_match_results.loc[(df_player_match_results['round'] >= round_number) & (df_player_match_results['feed_id'] == feed_id), :]
    player_rows['source'] = source
    if len(player_rows) > 0:
        df_player_match_results.update(player_rows)

df_player_match_results['source'] = "Draft"
df_transactions.apply(lambda row: get_transaction_status(row), axis=1)
df_player_match_results

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt,source
0,1.0,86078.0,10883.0,Lester,Love is Hind,368.0,MID,164.0,True,1009528,Tom,Green,GWS,MID,R1,Draft
1,2.0,86078.0,10883.0,Lester,Love is Hind,368.0,MID,106.0,True,1009528,Tom,Green,GWS,MID,R2,Draft
2,3.0,86078.0,10883.0,Lester,Love is Hind,368.0,MID,147.0,True,1009528,Tom,Green,GWS,MID,R3,Draft
3,4.0,86078.0,10883.0,Lester,Love is Hind,368.0,MID,92.0,True,1009528,Tom,Green,GWS,MID,R4,Draft
4,5.0,86078.0,10883.0,Lester,Love is Hind,368.0,MID,115.0,True,1009528,Tom,Green,GWS,MID,R5,Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,18.0,45168.0,14003.0,Luke,Gov Is In The Air,266.0,DEF,45.0,True,998260,Jack,Bowes,GCS,DEF,R18,Waiver
3308,16.0,10857.0,10836.0,Anthony,Cumming for the DB,708.0,MID,49.0,True,294557,Lachie,Hunter,WBD,MID,R16,Free Agent
3309,17.0,8258.0,11125.0,Simon,Sloane Andreas,708.0,MID,63.0,True,294557,Lachie,Hunter,WBD,MID,R17,Free Agent
3310,17.0,71052.0,10882.0,James,Take Me To Parish,152.0,MID,85.0,True,291720,Will,Hoskin-Elliott,COL,FWD MID,R17,Free Agent


In [31]:
df_player_match_results.to_csv('outputs/player_match_results.csv')